In [33]:
import pandas as pd
import yaml
import os
import plotly.express as px
import plotly
from plotly import graph_objects as go
import plotly.io as pio

pio.renderers.default = "vscode"
from numpy import dot
from numpy.linalg import norm

In [34]:
# TODO should german go as svo here?
# TODO add avg rank

In [35]:
full_name = {
    "pl": "Polish",
    "ru": "Russian",
    "uk": "Ukrainian",
    "slv": "Slovene",
    "cs": "Czech",
    "be": "Belarusian",
    "bg": "Bulgarian",
    "de": "German",
    "slk": "Slovak",
    "fr": "French",
    "ar": "Arabic",
    "id": "Indonesian",
    "en": "English",
    "fi": "Finnish",
    "zh": "Chinese",
    "lv": "Latvian",
    "es": "Spanish",
    "lt": "Lithuanian",
    "hr": "Croatian",
    "tr": "Turkish",
    "ko": "Korean",
    "hy": "Armenian",
    "fa": "Persian",
    "ja": "Japanese",
    "ga": "Irish",
    "cy": "Welsh",
    "nl": "Dutch",
}

In [36]:
lang_families = {'indo-european': ("pl", "be", "cs", "slk", "slv", "uk", "ru", "bg", "hr", "de", "fr", "en", "lv", "es", "lt", "hy", "fa", 'ga', 'cy', 'nl'), "afro-asiatic": ('ar', ), 'austronesian': ('id', ), 'uralic': ('fi',), 'sino-tibetan': ('zh', ), 'turkic': ('tr', ), 'koreanic': ('ko', ), 'japonic': ('ja', )}
lang_groups = {'slavic': ("pl", "be", "cs", "slk", "slv", "uk", "ru", "bg", "hr"), "germanic": ('de', "en", 'nl'), "italic/romance": ('fr', 'es'), 'semitic': ('ar', ), 'malay': ('id', ), 'finno-ugric': ('fi', ), 'sinitic': ('zh', ), 'baltic': ('lv', 'lt'), 'turkic': ('tr', ), 'koreanic': ('ko', ), 'armenian': ('hy', ), 'iranian': ('fa', ), 'japonic': ('ja', ), 'celtic': ('ga', 'cy')}

In [37]:
lang_to_size = {
    "pl": 1_301_917,
    "ru": 1_500_041,
    "uk": 825_591,
    "slv": 162_385,
    "cs": 413_512,
    "be": 157_651,
    "bg": 246_040,
    "de": 2_224_357,
    "slk": 227_255,
    "fr":  2_044_308,
    "ar": 618_436,
    "id": 440_545,
    "en": 5_725_914,
    "fi": 445_104,
    "zh": 1_024_062,
    "lv": 87_981,
    "es": 1_478_365,
    "lt": 189_673,
    "hr": 199_153,
    "tr": 316_491,
    "ko": 428_406,
    "hy": 246_231,
    "fa": 643_006,
    "ja": 1_122_821,
    "ga": 48_183,
    "cy": 101_401,
    "nl": 1_943_296,
}

In [38]:
len(full_name)

27

In [39]:
max_dict = {
    'ar': 6074,
    'pl': 17721,
    'bg': 8906,
    'be': 22852,
    'fr': 14448,
    'en': 12542,
    'lv': 12520,
    'fa': 4797,
    'lt': 2340,
    'cs': 68494,
    'es': 14286, 
    'zh': 3996,
    'slk': 8482,
    'slv': 10902,
    'uk': 5495,
    'ru': 69629,
    'de': 13813,
    'fi': 12216,
    'id': 4481,
    'hr': 6913,
    'tr': 14850,
    'ko': 23010,
    'hy': 1974,
    'fa': 26196,
    'ja': 7050,
    'ga': 4005,
    'cy': 1111,
    'nl': 12289,
}

In [40]:
lang_types = {'slavic': ("pl", "be", "cs", "slk", "slv", "uk", "ru", "bg", "hr"), 'svo': ("fr", "en", "lv", "lt", "es", "zh", "fi", "id", "de", "nl"), 'sov': ('tr', 'ko', 'hy', 'fa', 'ja'), 'vso': ('ar', 'ga', 'cy')}

In [41]:
has_cases = {True: (), False: ()}

In [42]:
lang_types_dict = {}
for lang_type, lang_list in lang_types.items():
    type_dict = {lan: lang_type for lan in lang_list}
    lang_types_dict.update(type_dict)

In [43]:
families_dict = {}
for lang_type, lang_list in lang_families.items():
    type_dict = {lan: lang_type for lan in lang_list}
    families_dict.update(type_dict)

In [44]:
group_dict = {}
for lang_type, lang_list in lang_groups.items():
    type_dict = {lan: lang_type for lan in lang_list}
    group_dict.update(type_dict)

In [45]:
RESULTS_CSV = 'multilingual-probing-visualization/probingOutputs/all_slavic/all_results.csv'

In [46]:
df = pd.read_csv(RESULTS_CSV)
df['train'] = df.train.str[2:-2]
df['test'] = df.test.str[2:-2]

In [47]:
assert set(df.train.unique()) == set(df.test.unique())
assert set(df.train.unique()) == set(max_dict.keys())
assert set(df.train.unique()) == set(full_name.keys())
assert set(df.train.unique()) == set(i for l in lang_types.values() for i in l)

In [48]:
df.tail()

,train,test,model,layer,rank,limit,seed,directory,dspear,uuas
6325,nl,zh,mBERT,6,128,1000.0,0,/home/amysiak/thesis/multilingual-probing-visu...,0.585347,0.418790
6326,nl,zh,mBERT,6,128,10000.0,0,/home/amysiak/thesis/multilingual-probing-visu...,0.643034,0.470045
6327,nl,zh,mBERT,6,128,2500.0,0,/home/amysiak/thesis/multilingual-probing-visu...,0.620485,0.444947
6328,nl,zh,mBERT,6,128,5000.0,0,/home/amysiak/thesis/multilingual-probing-visu...,0.640864,0.471584
6329,nl,zh,mBERT,6,128,7500.0,0,/home/amysiak/thesis/multilingual-probing-visu...,0.634353,0.456005


In [49]:
df.limit.unique()

array([  100.,  1000., 10000.,  2500.,  5000.,  7500.,    nan])

In [50]:
df["wiki_size_train"] = df.train.map(lang_to_size)
df["wiki_size_test"] = df.test.map(lang_to_size)

In [51]:
df = df[df.train.map(max_dict) > 7000]
df = df[df.test.map(max_dict) > 7000]

In [52]:
df = df[df.seed == 0]
df = df[df.wiki_size_train >= 200e3]
df = df[df.wiki_size_test >= 200e3]

In [53]:
def cos_sim(a, b):
    bad_indices_a  = [i for i, x in enumerate(a) if x == '--']
    bad_indices_b  = [i for i, x in enumerate(b) if x == '--']
    bad_indices = set(bad_indices_a) | set(bad_indices_b)
    a = [x for i, x in enumerate(a) if i not in bad_indices]
    b = [x for i, x in enumerate(b) if i not in bad_indices]
    return dot(a, b)/(norm(a)*norm(b))

In [54]:
df.limit.unique() # TODO replace nans with max, redo plots?

array([  100.,  1000., 10000.,  2500.,  5000.,  7500.,    nan])

In [55]:
df_avg = df.groupby(["train","test", "limit"]).mean()
df_avg = df_avg.reset_index()
df_avg.head()

/tmp/ipykernel_3808192/3170058264.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



,train,test,limit,layer,rank,seed,dspear,uuas,wiki_size_train,wiki_size_test
0,bg,bg,100.0,6.0,128.0,0.0,0.429731,0.349807,246040.0,246040.0
1,bg,bg,1000.0,6.0,128.0,0.0,0.717411,0.606988,246040.0,246040.0
2,bg,bg,2500.0,6.0,128.0,0.0,0.808573,0.774219,246040.0,246040.0
3,bg,bg,5000.0,6.0,128.0,0.0,0.830130,0.827339,246040.0,246040.0
4,bg,bg,7500.0,6.0,128.0,0.0,0.837259,0.846541,246040.0,246040.0


In [56]:
df_avg["lang_type"] = df_avg.train.map(lang_types_dict)

In [57]:
df_avg["lang_family"] = df_avg.train.map(families_dict)

In [58]:
df_avg["lang_group"] = df_avg.train.map(group_dict)

In [59]:
df_avg["wiki_size"] = df_avg.train.map(lang_to_size)

In [60]:
import lang2vec.lang2vec as l2v
geo_features = l2v.get_features(list(max_dict.keys()), "geo")
syntax_features = l2v.get_features(list(max_dict.keys()), "syntax_average", header=True)

In [61]:
syntax_features['en']

[1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.6667,
 1.0,
 1.0,
 0.3333,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.5,
 0.5,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.5,
 0.0,
 0.5,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0]

In [62]:
df_avg["geo_dist"] = df_avg.apply(lambda row: cos_sim(geo_features[row.train], geo_features[row.test]), axis=1)
df_avg["syntax_dist"] = df_avg.apply(lambda row: cos_sim(syntax_features[row.train], syntax_features[row.test]), axis=1)

/tmp/ipykernel_3808192/471460232.py:7: RuntimeWarning:

invalid value encountered in divide



In [63]:
def plot_lang_type_synt(lang_type):
    test_list = lang_types[lang_type]
    for test_lang in test_list:
        if len(df_avg[(df_avg.limit <= 10000) & (df_avg.test == test_lang)]) == 0:
            continue

        fig = px.scatter(data_frame=df_avg[(df_avg.limit <= 10000) & (df_avg.test == test_lang)], x="limit", y="uuas", hover_data=["train"], labels={"limit": "# train sentences", "uuas": "UUAS", lang_type: f"Is {lang_type}"}, title=f"Test: {full_name[test_lang]}, {lang_types_dict[test_lang]}", color="syntax_dist", color_discrete_sequence=px.colors.qualitative.Light24)
        
        fig.update_layout(
            autosize=False,
            width=800,
            height=600,
            yaxis_range=[0, 1],
            font={"size":20}
        )
        fig.show()

In [64]:
plot_lang_type_synt('svo')

In [65]:
from scipy.stats import spearmanr

In [66]:
df_avg_10k = df_avg[df_avg.limit == 10_000]
print(len(df_avg_10k))
df_avg_10k = df_avg_10k.dropna()
print(len(df_avg_10k))

180
168


In [67]:
spearmanr(df_avg_10k.geo_dist, df_avg_10k.uuas)

SpearmanrResult(correlation=0.5884337567614335, pvalue=4.96268768926208e-17)

In [71]:
spearmanr(df_avg_10k.syntax_dist, df_avg_10k.uuas)

SpearmanrResult(correlation=0.657088477672405, pvalue=3.9452513874108145e-22)

In [72]:
def plot_lang_type(lang_type, switch=False, family=True):
    if family:
        other_col = "lang_family"
        other_dict = families_dict
    else:
        other_col = "lang_group"
        other_dict = group_dict
    test_list = lang_types[lang_type]
    for test_lang in test_list:
        if len(df_avg[(df_avg.limit <= 10000) & (df_avg.test == test_lang)]) == 0:
            continue
        if not switch:
            fig = px.scatter(data_frame=df_avg[(df_avg.limit <= 10000) & (df_avg.test == test_lang)], x="limit", y="uuas", symbol=other_col, hover_data=["train"], labels={"limit": "# train sentences", "uuas": "UUAS", lang_type: f"Is {lang_type}"}, title=f"Test: {full_name[test_lang]}, {lang_types_dict[test_lang]}, {other_dict[test_lang]}", color="lang_type", color_discrete_sequence=px.colors.qualitative.Light24)
        else:
            fig = px.scatter(data_frame=df_avg[(df_avg.limit <= 10000) & (df_avg.test == test_lang)], x="limit", y="uuas", color=other_col, hover_data=["train"], labels={"limit": "# train sentences", "uuas": "UUAS", lang_type: f"Is {lang_type}"}, title=f"Test: {full_name[test_lang]}, {lang_types_dict[test_lang]}, {other_dict[test_lang]}", symbol="lang_type", color_discrete_sequence=px.colors.qualitative.Light24)
        fig.update_layout(
            autosize=False,
            width=800,
            height=600,
            yaxis_range=[0, 1],
            font={"size":20}
        )
        fig.show()

In [73]:
# klastrowanie w 2D które da się pokolorować?
# dbscan, każdy punkt to język, odległość - średnia wyniku (test, train) i (train, test)
# albo użyć wyników jako współrzędnych, tj. każdy punkt to język testowy, współrzędne to trainy

# regresja liniowa przewidująca wyniki? jaki procent wyników da się przewidzieć z szyku zdania?

In [74]:
plot_lang_type("svo")

In [75]:
plot_lang_type("slavic")

In [76]:
plot_lang_type("svo", switch=True)

In [77]:
plot_lang_type("svo", family=False)

In [78]:
plot_lang_type("svo", family=False, switch=True)

In [79]:
plot_lang_type("slavic")

In [80]:
plot_lang_type("sov")

In [81]:
plot_lang_type("vso")

In [82]:
df[df.limit == 10_000].groupby('train').mean().sort_values(by='uuas', ascending=False)

/tmp/ipykernel_3808192/1953500136.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



,layer,rank,limit,seed,dspear,uuas,wiki_size_train,wiki_size_test
train,,,,,,,,
de,6.0,128.0,10000.0,0.0,0.755511,0.689631,2224357.0,1.335172e+06
nl,6.0,128.0,10000.0,0.0,0.760332,0.678563,1943296.0,1.337389e+06
en,6.0,128.0,10000.0,0.0,0.741870,0.670761,5725914.0,1.335172e+06
fr,6.0,128.0,10000.0,0.0,0.750397,0.665331,2044308.0,1.335172e+06
pl,6.0,128.0,10000.0,0.0,0.755575,0.662949,1301917.0,1.335172e+06
ru,6.0,128.0,10000.0,0.0,0.752734,0.661656,1500041.0,1.335172e+06
cs,6.0,128.0,10000.0,0.0,0.753940,0.660709,413512.0,1.335172e+06
es,6.0,128.0,10000.0,0.0,0.748868,0.647610,1478365.0,1.335172e+06
fa,6.0,128.0,10000.0,0.0,0.731858,0.620839,643006.0,1.337389e+06


In [83]:
df["train_lang_type"] = df.train.map(lang_types_dict)
df["test_lang_type"] = df.test.map(lang_types_dict)
df[(df.limit == 10_000) & (df.test_lang_type != 'slavic')].groupby('train').mean().sort_values(by='uuas', ascending=False)

/tmp/ipykernel_3808192/4129219901.py:3: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



,layer,rank,limit,seed,dspear,uuas,wiki_size_train,wiki_size_test
train,,,,,,,,
de,6.0,128.0,10000.0,0.0,0.758411,0.675727,2224357.0,1637206.8
en,6.0,128.0,10000.0,0.0,0.744359,0.655264,5725914.0,1637206.8
nl,6.0,128.0,10000.0,0.0,0.747122,0.648744,1943296.0,1637206.8
fr,6.0,128.0,10000.0,0.0,0.746092,0.648725,2044308.0,1637206.8
es,6.0,128.0,10000.0,0.0,0.742139,0.628169,1478365.0,1637206.8
ru,6.0,128.0,10000.0,0.0,0.741151,0.615646,1500041.0,1637206.8
cs,6.0,128.0,10000.0,0.0,0.741406,0.615281,413512.0,1637206.8
pl,6.0,128.0,10000.0,0.0,0.739854,0.612470,1301917.0,1637206.8
fa,6.0,128.0,10000.0,0.0,0.724259,0.598633,643006.0,1637206.8


In [84]:
import lang2vec.lang2vec as l2v
geo_features = l2v.get_features(list(max_dict.keys()), "geo")

Are Latvian and Finnish slavic as well?? both have cases, ha

what about Bulgarian then??

Persian has a variable word order, apparently

Maybe language families?

Dataset size: wikipedia sizes are here: https://meta.wikimedia.org/w/index.php?title=List_of_Wikipedias/Table&oldid=18434392

Article with cased mBERT was submitted in October 2018

German and Dutch are considered SVO in conventional typology and SOV in generative grammar. They can be considered SOV but with V2 word order as an overriding rule for the finite verb in main clauses, which results in SVO in some cases and SOV in others.

https://en.wikipedia.org/wiki/Subject%E2%80%93object%E2%80%93verb_word_order

Other hypotheses:
* UD data quality/size?
* ???